Random Forest Algorithm

## Bootstraping

> We first split the features in to different datasets, Then we create Decision trees for eachof the dataset

> Next we introduce the data to all the trees and get the prediction


> Next we get the majority voting

## Aggregation

> Combining results from multiple models

---
Combination of two of above task called bagging

why we use bootstraping and feature selection?

We use bootstraping for randomize and prevent overfitting.
Other than like a single Decision tree the random forest has multiple trees so that it is suit for numeric datasets

> Importing nessasary Librries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

> Reading the dataset(After EDA)

In [2]:
final_df=pd.read_csv("./final_heart_last.csv")

Dividing the training sets and test sets

In [3]:
from sklearn.model_selection import train_test_split\

X = final_df.drop('HeartDisease', axis=1)
y = final_df['HeartDisease']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, test_size=.3, random_state=42)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(567, 19) (244, 19)
(567,) (244,)


Importing the RandomForest Model

In [4]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier

In [5]:
RandomForest = RandomForestClassifier()

> This is the random forest values without Tuning and using the basic model of scikit learn

In [6]:
acc_score = []
roc_score = []
f1 = []
name_model = []


# Training the Model
RandomForest.fit(X_train,y_train)

#Setting the Prediction value form the RandomForest
y_pred =  RandomForest.predict(X_test)

#Testing the accuracy of the Model
acc_score.append(accuracy_score(y_test, y_pred))

#Testing the ROC
roc_score.append(roc_auc_score(y_test, y_pred))

#Tesing the F1 score
f1.append(f1_score(y_test, y_pred, average='weighted'))

name_model.append(type(RandomForest).__name__)

result = pd.DataFrame(
    {'Model Name' : name_model,
     'accuracy': acc_score,
     'roc auc' : roc_score,
     'f1-weighted' : f1}
)
result.sort_values('f1-weighted',ascending=False)

,Model Name,accuracy,roc auc,f1-weighted
0,RandomForestClassifier,0.852459,0.851538,0.85232


Hyperparameter Tuning using the Grid Search

> Setting the Parameter grid

> This is used to set what typical values we can use for tuning

In [7]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'log2'],
    'bootstrap': [True, False],
    'class_weight': [None, 'balanced'],
    'criterion': ['gini', 'entropy']
}

> This is the implementation of grid search. In this we can check model performance for various parameter values without tuning manually.

In [8]:
# Initialize the RandomForestClassifier
rf = RandomForestClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           scoring=['accuracy', 'roc_auc', 'f1'],
                           refit='roc_auc',  # Refitting based on F1-weighted score
                           cv=5,  # 5-fold cross-validation
                           verbose=5, n_jobs=-1)



### Checking values

In [9]:
# Fit the grid search to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, None],
                         'max_features': ['auto', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             refit='roc_auc', scoring=['accuracy', 'roc_auc', 'f1'], verbose=5)

> for getting the best estimater model parameters

In [10]:
print(grid_search.best_estimator_)


RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_features='log2', min_samples_leaf=2,
                       min_samples_split=5)


### Saving the model for demonstration

In [12]:
from joblib import dump, load
dump(RandomForestModel, 'random_forest_model.joblib')

['random_forest_model.joblib']

In [13]:
print(grid_search.best_params_)

{'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}


In [14]:
print(grid_search.best_score_)

0.9303727481976068


### For further checking of the values that grid search has obtained

In [15]:
df2 = pd.DataFrame(grid_search.cv_results_)
df2 = df2.sort_values("rank_test_roc_auc")
df2.to_csv("cv_results.csv")

In [44]:
RandomForestModel = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_features='log2', min_samples_leaf=2,
                       min_samples_split=5)

## This is the values gained after Tuning and Cross validation

In [42]:
name_model = []
name_model.append(type(RandomForestModel).__name__)
# Assuming you have already performed GridSearchCV and have the scores
acc_score_cv = grid_search.cv_results_['mean_test_accuracy']
roc_auc_score_cv = grid_search.cv_results_['mean_test_roc_auc']
f1_score_cv = grid_search.cv_results_['mean_test_f1']

# Create a DataFrame to display the cross-validated results
cv_results_df = pd.DataFrame({
    'mean_test_accuracy': acc_score_cv,
    'mean_test_roc_auc': roc_auc_score_cv,
    'mean_test_f1': f1_score_cv
})
Final_result_cv = cv_results_df.sort_values('mean_test_roc_auc',ascending=False).reset_index(drop=True).head(1)
Final_result_cv.insert(0, 'Model Name', name_model)
Final_result_cv

,Model Name,mean_test_accuracy,mean_test_roc_auc,mean_test_f1
0,RandomForestClassifier,0.850116,0.930373,0.861886


In [45]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split\

acc_score = []
roc_score = []
f1 = []
name_model = []


# Training the Model
RandomForestModel.fit(X_train,y_train)

#Setting the Prediction value form the RandomForest
y_pred =  RandomForestModel.predict(X_test)

#Testing the accuracy of the Model
acc_score.append(accuracy_score(y_test, y_pred))
#Testing the ROC
roc_score.append(roc_auc_score(y_test, y_pred))
#Tesing the F1 score
f1.append(f1_score(y_test, y_pred))
name_model.append(type(RandomForestModel).__name__)

result = pd.DataFrame(
    {'Model Name' : name_model,
     'accuracy': acc_score,
     'roc auc' : roc_score,
     'f1 score' : f1}
)
result.sort_values('f1 score',ascending=False)

,Model Name,accuracy,roc auc,f1 score
0,RandomForestClassifier,0.852459,0.852211,0.858268
